# System Identification

This notebook describes MuJoCo's [system identification](https://en.wikipedia.org/wiki/System_identification) framework. System identification optimizes parameters to make a simulation match measurements. We introduce the framework's core concepts and walk through some basic examples. More detailed exercises will be added soon.

**In this notebook we will:**

1. [**Formulation:**](#formulation) briefly introduce the sysid problem
2. [**Core Concepts:**](#core-concepts) estimate the mass of a mass-spring-damper
3. [**API:**](#api) tour core API concepts like `Parameter`, `TimeSeries`, and `ModelSequences`
4. [**Robot arm:**](#robot-arm) identify joint armature on a 5-DOF arm, with confidence intervals, sensor plots, and an interactive HTML report
5. [**Parameter Identifiability:**](#ambiguity) understand and diagnose when parameters are unidentifiable

# Setup

In [ ]:
#@title Install MuJoCo and mediapy
!pip install -q mujoco[sysid] --pre -f https://py.mujoco.org/
!pip install -q mediapy

In [ ]:
#@title Set up GPU rendering
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

In [ ]:
#@title Configure EGL backend
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

In [ ]:
import mujoco
import mujoco.rollout as rollout
from mujoco import sysid
import numpy as np
import matplotlib.pyplot as plt
import mediapy as media
from absl import logging
import base64
from IPython.display import IFrame

logging.set_verbosity("INFO")

def display_report(report):
    html_b64 = base64.b64encode(report.build().encode()).decode()
    return IFrame(src=f"data:text/html;base64,{html_b64}", width="100%", height=800)

<a name="formulation"></a>
# Formulation

This library does [**gray-box**](https://en.wikipedia.org/wiki/Grey_box_model) identification: you supply the model structure
(rigid-body dynamics, contacts, actuators) via a MuJoCo XML, and the optimizer
adjusts the parameters you designate as unknown.

Given $K$ parameters collected in a vector $\theta$ and $N$ sensor
measurements $y$, we simulate the model to produce predicted outputs
$\bar y(\theta)$ and minimize the weighted residual:

$$\min_\theta \; \tfrac{1}{2}\lVert W\bigl(\bar y(\theta) - y\bigr)\rVert^2
\qquad \text{s.t.}\quad l \preccurlyeq \theta \preccurlyeq u$$

This is a box-constrained **nonlinear least-squares** problem. The optimizer
uses a Gauss-Newton / Levenberg-Marquardt algorithm with finite-difference
Jacobians. Each parameter perturbation requires an independent simulation
rollout, and all of them execute in a single batched call to [`mujoco.rollout`](https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/rollout.ipynb),
parallelized across CPU threads.

For a detailed treatment of the underlying optimizer, see the
[Least Squares](https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/least_squares.ipynb) notebook.

<a name="core-concepts"></a>
# 1. Core Concepts

We illustrate the framework's core concepts with an actuated [mass-spring-damper](https://en.wikipedia.org/wiki/Mass-spring-damper_model). We know the exact spring stiffness and damping, but the mass must be estimated.

### The model

A box on a spring, driven by a force actuator, with position and velocity
sensors. The true mass is **1.0 kg**.

In [ ]:
#@title SPRING_MASS_XML { vertical-output: true}
SPRING_MASS_XML = """\
<mujoco model="spring_mass">
  <option timestep="0.002">
    <flag contact="disable"/>
  </option>
  <worldbody>
    <body name="ball" pos="0 0 0.1">
      <inertial pos="0 0 0" mass="1.0" diaginertia="0.001 0.001 0.001"/>
      <joint name="slide" type="slide" axis="1 0 0"
             stiffness="100" damping="5.0"/>
      <geom type="box" size="0.05 0.05 0.05"/>
    </body>
  </worldbody>
  <actuator>
    <motor name="push" joint="slide"/>
  </actuator>
  <sensor>
    <jointpos name="position" joint="slide"/>
    <jointvel name="velocity" joint="slide"/>
  </sensor>
</mujoco>
"""

### Generate "measured" data

We simulate the true model to create our ground-truth sensor recordings.
A multi-frequency excitation signal produces a non-trivial trajectory.
<!-- ensures the system is well-excited across its dynamics. -->

`create_initial_state` packs the joint positions, velocities and actuator
activations into the
[state vector](https://mujoco.readthedocs.io/en/stable/computation/index.html#the-state)
that `mujoco.rollout` expects. The results are wrapped in `TimeSeries`
objects, which pair timestamps with data columns and are covered in detail
in [Section 2](#api).

In [ ]:
spec = mujoco.MjSpec.from_string(SPRING_MASS_XML)
model = spec.compile()
data = mujoco.MjData(model)

duration = 3.0
n_steps = int(duration / model.opt.timestep)
t = np.arange(n_steps) * model.opt.timestep

ctrl = (5.0 * np.sin(2 * np.pi * 1.5 * t)
        + 3.0 * np.sin(2 * np.pi * 3.7 * t)).reshape(-1, 1)

initial_state = sysid.create_initial_state(model, data.qpos, data.qvel, data.act)

state, sensor = rollout.rollout(model, data, initial_state, ctrl[:-1])
state = np.squeeze(state, axis=0)
sensor = np.squeeze(sensor, axis=0)
times = state[:, 0]

Let's visualize the result:

In [ ]:
#@title { vertical-output: true}

# Render the rollout.
frames = sysid.render_rollout(
    model, data, state[None], framerate=30, height=400, width=560
)
media.show_video(frames, fps=30)

control_ts = sysid.TimeSeries(t, ctrl)
sensor_ts = sysid.TimeSeries.from_names(times, sensor, model)

fig, axes = plt.subplots(3, 1, figsize=(5, 4), sharex=True,
                         gridspec_kw={"height_ratios": [2, 2, 1]})

axes[0].plot(times, sensor[:, 0], color="C0", linewidth=1.0)
axes[0].set_ylabel("Position (m)")
axes[0].set_title("Measured trajectory (true mass = 1.0 kg)")

axes[1].plot(times, sensor[:, 1], color="C1", linewidth=1.0)
axes[1].set_ylabel("Velocity (m/s)")

axes[2].plot(t, ctrl[:, 0], color="0.4", linewidth=0.8)
axes[2].set_ylabel("Control (N)")
axes[2].set_xlabel("Time (s)")

for ax in axes:
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Construct the System Identification Problem

#### Define the unknown parameter

A `Parameter` wraps a scalar (or vector) value with:
- **bounds** for box-constrained optimization
- a **modifier** callback that stamps the current value onto an [`MjSpec`](https://mujoco.readthedocs.io/en/stable/python.html#model-editing)

The `nominal` value is a fixed reference point (here we set it to the true
value for later comparison, but in practice it would be your best prior
guess). The mutable `value` is what the optimizer actually updates.

We'll start the optimizer at **mass = 2.0 kg**, double the true value.

In [ ]:
def set_mass(spec, param):
    """Modifier: stamp the current mass value onto the MjSpec."""
    spec.body("ball").mass = param.value[0]

params = sysid.ParameterDict()
params.add(sysid.Parameter(
    "mass",
    nominal=1.0,
    min_value=0.3,
    max_value=3.0,
    modifier=set_mass,
))

params["mass"].value[:] = 2.0
print(f"Starting mass: {params['mass'].value[0]:.2f} kg  (true: 1.0 kg)")

#### Package data

`ModelSequences` bundles an `MjSpec` with one or more measured trajectories
(initial state, controls and sensor readings).

In [ ]:
ms = sysid.ModelSequences(
    "spring_mass", spec, "measured", initial_state, control_ts, sensor_ts,
)

#### Optimize

`build_residual_fn` creates a callable that, for a given parameter vector,
applies the parameters to the spec, rolls out the simulation, and returns the
difference between predicted and measured sensor data. This difference is the
**residual** that the optimizer drives to zero.

In [ ]:
residual_fn = sysid.build_residual_fn(models_sequences=[ms])

opt_params, opt_result = sysid.optimize(
    initial_params=params,
    residual_fn=residual_fn,
    optimizer="mujoco",
    verbose=True,
)

print(f"\nRecovered mass: {opt_params['mass'].value[0]:.4f} kg  (true: 1.0 kg)")

#### Report

Inspect the report to see that the optimized predictions fit the nominal (ground truth) while the initial sensor predictions do not.

The report also contains measurement comparisons, parameter tables, confidence intervals, and other debugging information.

In [ ]:
report = sysid.default_report(
    models_sequences=[ms],
    initial_params=params,
    opt_params=opt_params,
    residual_fn=residual_fn,
    opt_result=opt_result,
    title="Mass Spring Damper Identification",
    generate_videos=False,
)
display_report(report)

<a name="api"></a>
# 2. API

The warm-up introduced `Parameter`, `TimeSeries`, and `ModelSequences` in
passing. Here we look at each one more closely.

## ParameterDict

In the warm-up we created a single `Parameter` with a name, bounds and a
modifier callback. A `ParameterDict` groups multiple parameters for the optimizer.

Parameters can be **frozen**: they are excluded from the optimization vector
but their modifiers are still applied at the frozen value.

In [ ]:
# Three parameters for the spring-mass model. Damping is frozen.
params = sysid.ParameterDict()
params.add(sysid.Parameter(
    "mass", nominal=1.0, min_value=0.3, max_value=3.0,
    modifier=lambda s, p: setattr(s.body("ball"), "mass", p.value[0]),
))
params.add(sysid.Parameter(
    "stiffness", nominal=100.0, min_value=30.0, max_value=300.0,
    modifier=lambda s, p: setattr(s.joint("slide"), "stiffness", p.value[0]),
))
params.add(sysid.Parameter(
    "damping", nominal=5.0, min_value=0.0, max_value=20.0, frozen=True,
    modifier=lambda s, p: setattr(s.joint("slide"), "damping", p.value[0]),
))

print("Parameter vector (excludes frozen):", params.as_vector())
print("Bounds:", params.get_bounds())
print("Frozen 'damping' still in dict:    ", params["damping"].value)

## TimeSeries

`TimeSeries` is an immutable container for timestamped signals. The
`from_names` factory that we used in the warm-up automatically creates a
**signal mapping**, a dict from sensor names to column indices. This mapping
lets the library match predicted signals to measured ones by name.

`TimeSeries` also supports resampling to a different timestep, which is useful
when your measured data and simulation run at different rates.

In [ ]:
# sensor_ts was created in the warm-up with from_names
print("Signal mapping:", sensor_ts.signal_mapping)

# Resample to a coarser timestep (returns a new TimeSeries)
ts_coarse = sensor_ts.resample(target_dt=0.01)
print(f"Original: {len(sensor_ts.times)} pts  ->  Resampled: {len(ts_coarse.times)} pts")

## ModelSequences

`ModelSequences` bundles an `MjSpec` with one or more measured trajectories
(initial state, controls and sensor readings). It is the input to
`build_residual_fn`.

You can pass **multiple trajectories** for the same spec. This is the key
mechanism for improving parameter identifiability, as we'll see in the next
section.

In [ ]:
# Single trajectory (as in the warm-up)
ms_single = sysid.ModelSequences(
    "spring_mass", spec, "traj_0", initial_state, control_ts, sensor_ts,
)
print(f"Single trajectory: {ms_single.sequence_name}")

# Multiple trajectories for the same spec -- just pass lists
ms_multi = sysid.ModelSequences(
    "spring_mass", spec,
    ["traj_0", "traj_1"],           # names
    [initial_state, initial_state], # initial states
    [control_ts, control_ts],       # controls
    [sensor_ts, sensor_ts],         # sensor data
)
print(f"Multiple trajectories: {ms_multi.sequence_name}")

<a name="robot-arm"></a>
# 4. Robot Arm: Identifying Joint Armature

Now we tackle a realistic problem. A 5-DOF robot arm is driven by motor actuators that apply joint torques, and the armature is unknown. Armature represents reflected rotor inertia through the gear train. It is a common source of sim-to-real gap. Getting it wrong means the simulated arm accelerates too fast or too slow under the same applied torque. You can read more about it [here](https://mujoco.readthedocs.io/en/stable/XMLreference.html#body-joint-armature).

### Arm model

Five hinge joints with motor actuators (torque control). Joint damping
provides passive dissipation. The true armature values decrease from base
to tip, reflecting smaller motors on distal joints. Position sensors on every joint.

In [ ]:
#@title ARM_XML { vertical-output: true}
ARM_XML = """\
<mujoco model="arm">
  <compiler angle="radian" autolimits="true"/>
  <option integrator="implicitfast" timestep="0.002">
    <flag contact="disable"/>
  </option>
  <worldbody>
    <body name="link1" pos="0 0 0.1">
      <inertial pos="0 0 0.05" mass="1.0" diaginertia="0.01 0.01 0.005"/>
      <joint name="joint1" type="hinge" axis="0 0 1" range="-3.14 3.14"
             armature="0.5" damping="1.0"/>
      <geom type="capsule" fromto="0 0 0 0 0 0.1" size="0.04"/>
      <body name="link2" pos="0 0 0.1">
        <inertial pos="0 0 0.05" mass="0.8" diaginertia="0.008 0.008 0.004"/>
        <joint name="joint2" type="hinge" axis="0 1 0" range="-3.14 3.14"
               armature="0.4" damping="0.8"/>
        <geom type="capsule" fromto="0 0 0 0 0 0.1" size="0.035"/>
        <body name="link3" pos="0 0 0.1">
          <inertial pos="0 0 0.05" mass="0.6" diaginertia="0.006 0.006 0.003"/>
          <joint name="joint3" type="hinge" axis="0 1 0" range="-3.14 3.14"
                 armature="0.3" damping="0.6"/>
          <geom type="capsule" fromto="0 0 0 0 0 0.1" size="0.03"/>
          <body name="link4" pos="0 0 0.1">
            <inertial pos="0 0 0.04" mass="0.4" diaginertia="0.004 0.004 0.002"/>
            <joint name="joint4" type="hinge" axis="0 0 1" range="-3.14 3.14"
                   armature="0.2" damping="0.4"/>
            <geom type="capsule" fromto="0 0 0 0 0 0.08" size="0.025"/>
            <body name="link5" pos="0 0 0.08">
              <inertial pos="0 0 0.03" mass="0.2" diaginertia="0.002 0.002 0.001"/>
              <joint name="joint5" type="hinge" axis="0 1 0" range="-3.14 3.14"
                     armature="0.1" damping="0.2"/>
              <geom type="capsule" fromto="0 0 0 0 0 0.06" size="0.02"/>
            </body>
          </body>
        </body>
      </body>
    </body>
  </worldbody>
  <actuator>
    <motor name="act1" joint="joint1"/>
    <motor name="act2" joint="joint2"/>
    <motor name="act3" joint="joint3"/>
    <motor name="act4" joint="joint4"/>
    <motor name="act5" joint="joint5"/>
  </actuator>
  <sensor>
    <jointpos name="joint1_pos" joint="joint1"/>
    <jointpos name="joint2_pos" joint="joint2"/>
    <jointpos name="joint3_pos" joint="joint3"/>
    <jointpos name="joint4_pos" joint="joint4"/>
    <jointpos name="joint5_pos" joint="joint5"/>
  </sensor>
</mujoco>
"""

JOINT_NAMES = ["joint1", "joint2", "joint3", "joint4", "joint5"]
TRUE_ARMATURE = {"joint1": 0.5, "joint2": 0.4, "joint3": 0.3,
                 "joint4": 0.2, "joint5": 0.1}

### Generate measured data

We simulate the true model and add sensor noise to mimic real encoder
readings. A multi-frequency torque excitation ensures each joint is
well-excited. With motor actuators the control signal **is** the torque,
so there is no feedback loop to mask the effect of armature.

In [ ]:
spec = mujoco.MjSpec.from_string(ARM_XML)
model = spec.compile()
data = mujoco.MjData(model)

duration = 2.0
n_steps = int(duration / model.opt.timestep)
t = np.arange(n_steps) * model.opt.timestep

# Sinusoidal torques with different frequency and amplitude per joint
ctrl = np.column_stack([
    5.0 * np.sin(2 * np.pi * 0.5 * t),
    4.0 * np.sin(2 * np.pi * 0.7 * t + 0.5),
    3.0 * np.sin(2 * np.pi * 0.4 * t + 1.0),
    2.0 * np.sin(2 * np.pi * 0.9 * t + 1.5),
    1.0 * np.sin(2 * np.pi * 0.6 * t + 2.0),
])

initial_state = sysid.create_initial_state(model, data.qpos, data.qvel, data.act)
state, sensor = rollout.rollout(model, data, initial_state, ctrl[:-1])
state = np.squeeze(state, axis=0)
sensor = np.squeeze(sensor, axis=0)
times = state[:, 0]

# Add realistic sensor noise
rng = np.random.default_rng(seed=0)
noise_std = np.zeros(sensor.shape[1])
noise_std[:] = 0.6   # position noise (rad)
sensor_noisy = sensor + rng.normal(scale=noise_std, size=sensor.shape)

control_ts = sysid.TimeSeries(t, ctrl)
sensor_ts = sysid.TimeSeries.from_names(times, sensor_noisy, model)

print(f"Sensor channels: {sensor.shape[1]}  "
      f"({model.nsensor} sensors: 5 pos + 5 vel)")
print(f"Timesteps: {len(times)} ({duration}s at dt={model.opt.timestep})")
print(f"Noise std: {noise_std[:5][0]:.0e} rad (pos)")

### Define armature parameters

One scalar parameter per joint. The modifier callback sets the joint's `armature` attribute on the spec. We start from a wrong initial guess of 0.01 for all joints, which is up to **50x too small** for the base joint.

In [ ]:
INIT_ARMATURE = 0.01

def make_armature_modifier(joint_name):
    """Create a modifier that sets armature on a specific joint."""
    def modifier(spec, param):
        spec.joint(joint_name).armature = param.value[0]
    return modifier

params = sysid.ParameterDict()
for name in JOINT_NAMES:
    true_val = TRUE_ARMATURE[name]
    params.add(sysid.Parameter(
        f"{name}_armature",
        nominal=true_val,
        min_value=0.01,
        max_value=0.6,
        modifier=make_armature_modifier(name),
    ))
    # Start from wrong initial guess
    params[f"{name}_armature"].value[:] = INIT_ARMATURE

print("Initial parameter vector:", params.as_vector())
print("True values:             ",
      np.array([TRUE_ARMATURE[n] for n in JOINT_NAMES]))

### Optimize

In [ ]:
#@title { vertical-output: true}
ms = sysid.ModelSequences(
    "arm", spec, "sinusoidal", initial_state, control_ts, sensor_ts,
)

residual_fn = sysid.build_residual_fn(models_sequences=[ms])

opt_params, opt_result = sysid.optimize(
    initial_params=params,
    residual_fn=residual_fn,
    optimizer="mujoco",
)

#### Report

**Confidence Internals.** Because the measured data contains sensor noise, the optimal residuals have variance, which can be used to estimate 95% parameter confidence intervals. These are displayed in the report "Parameter Distribution" section.

In [ ]:
report = sysid.default_report(
    models_sequences=[ms],
    initial_params=params,
    opt_params=opt_params,
    residual_fn=residual_fn,
    opt_result=opt_result,
    title="Robot Arm Armature Identification",
    generate_videos=False,
)
display_report(report)

### Rendered overlay: initial vs. optimized vs. ground truth

`render_rollout` takes a list of models and a batch of state trajectories
and renders them into a single scene, which lets us visually compare how
different parameter values affect the motion. We render two side-by-side
videos:
- **Before:** initial guess (red) vs. ground truth (green)
- **After:** optimized (blue) vs. ground truth (green)

In [ ]:
#@title { vertical-output: true}
def set_body_rgba(body, rgba):
    """Recursively set rgba on all geoms in a body tree."""
    for geom in body.geoms:
        geom.rgba = rgba
    for child in body.bodies:
        set_body_rgba(child, rgba)

def make_colored_model(base_spec, rgba, armature_values):
    """Copy spec, set geom colors and armature, compile."""
    s = base_spec.copy()
    for name, val in armature_values.items():
        s.joint(name).armature = val
    set_body_rgba(s.worldbody, rgba)
    return s.compile()

true_armature = {n: TRUE_ARMATURE[n] for n in JOINT_NAMES}
init_armature = {n: INIT_ARMATURE for n in JOINT_NAMES}
opt_armature = {n: opt_params[f"{n}_armature"].value[0] for n in JOINT_NAMES}

green = [0.2, 0.8, 0.2, 0.7]
red = [1.0, 0.2, 0.2, 0.7]
blue = [0.2, 0.4, 1.0, 0.7]

truth_model = make_colored_model(spec, green, true_armature)
init_model = make_colored_model(spec, red, init_armature)
opt_model = make_colored_model(spec, blue, opt_armature)

fps = 30

# Before: initial (red) vs ground truth (green)
models_before = [init_model, truth_model]
datas_before = [mujoco.MjData(m) for m in models_before]
state_before, _ = rollout.rollout(
    models_before, datas_before, initial_state, ctrl[:-1]
)
frames_before = sysid.render_rollout(
    models_before, datas_before[0], state_before,
    framerate=fps, height=400, width=560,
)

# After: optimized (blue) vs ground truth (green)
models_after = [opt_model, truth_model]
datas_after = [mujoco.MjData(m) for m in models_after]
state_after, _ = rollout.rollout(
    models_after, datas_after, initial_state, ctrl[:-1]
)
frames_after = sysid.render_rollout(
    models_after, datas_after[0], state_after,
    framerate=fps, height=400, width=560,
)

# Side by side
media.show_videos(
    [frames_before, frames_after],
    fps=fps,
    titles=["Before: initial (red) vs truth (green)",
            "After: optimized (blue) vs truth (green)"],
)

<a name="ambiguity"></a>
# 4. Parameter Identifiability

Sometimes a single experiment cannot uniquely determine all parameters. This is related to the concept of [Structural Identifiability](https://en.wikipedia.org/wiki/Structural_identifiability).

Consider a cart driven by a motor:

$$m\,\ddot x = b\,u(t)$$

where $m$ is the cart mass and $b$ is the torque constant ([`gear`](https://mujoco.readthedocs.io/en/stable/XMLreference.html#actuator-general-gear)). The response
depends **only on the ratio** $b/m$. Doubling both $b$ and $m$ produces an
identical trajectory. With a single recording, the individual values are
fundamentally unidentifiable.

We're going to visualize the ambiguity and resolve it by optimizing over a **second trajectory** recorded with a known mass perturbation.

In [ ]:
#@title CART_XML { vertical-output: true}
CART_XML = """\
<mujoco model="cart">
  <option timestep="0.002">
    <flag contact="disable"/>
  </option>
  <worldbody>
    <body name="cart" pos="0 0 0.1">
      <inertial pos="0 0 0" mass="2.0" diaginertia="0.001 0.001 0.001"/>
      <joint name="slide" type="slide" axis="1 0 0"/>
      <geom type="sphere" size="0.05"/>
    </body>
  </worldbody>
  <actuator>
    <motor name="motor" joint="slide" gear="3.0"/>
  </actuator>
  <sensor>
    <jointpos name="position" joint="slide"/>
    <jointvel name="velocity" joint="slide"/>
  </sensor>
</mujoco>
"""

TRUE_MASS = 2.0
TRUE_GEAR = 3.0
PAYLOAD_MASS = 1.0

# Base cart
spec_base = mujoco.MjSpec.from_string(CART_XML)

# Payload variant: add a rigid mass to the cart body
spec_pay = mujoco.MjSpec.from_string(CART_XML)
payload = spec_pay.body("cart").add_body()
payload.name = "payload"
payload.mass = PAYLOAD_MASS
payload.inertia = [0.001, 0.001, 0.001]

### Generate data for both configurations

In [ ]:
#@title generate_data() { vertical-output: true}
def generate_data(spec, duration=0.6):
    """Rollout a spec and return (control_ts, sensor_ts, initial_state)."""
    model = spec.compile()
    data = mujoco.MjData(model)

    n_steps = int(duration / model.opt.timestep)
    t = np.arange(n_steps) * model.opt.timestep

    ctrl = (3.0 * np.sin(2 * np.pi * 2.0 * t)
            + 2.0 * np.sin(2 * np.pi * 5.0 * t)).reshape(-1, 1)

    initial_state = sysid.create_initial_state(
        model, data.qpos, data.qvel, data.act
    )
    state, sensor = rollout.rollout(model, data, initial_state, ctrl[:-1])
    state = np.squeeze(state, axis=0)
    sensor = np.squeeze(sensor, axis=0)
    times = state[:, 0]

    control_ts = sysid.TimeSeries(t, ctrl)
    sensor_ts = sysid.TimeSeries.from_names(times, sensor, model)
    return control_ts, sensor_ts, initial_state


ctrl_base, sens_base, state0_base = generate_data(spec_base)
ctrl_pay, sens_pay, state0_pay = generate_data(spec_pay)

### Define parameters: mass and gear

In [ ]:
def set_cart_mass(spec, param):
    spec.body("cart").mass = param.value[0]

def set_gear_ratio(spec, param):
    spec.actuator("motor").gear[0] = param.value[0]

def make_params(mass_init, gear_init):
    """Create a ParameterDict with given starting values."""
    params = sysid.ParameterDict()
    params.add(sysid.Parameter(
        "mass", nominal=TRUE_MASS, min_value=0.5, max_value=5.0,
        modifier=set_cart_mass,
    ))
    params.add(sysid.Parameter(
        "gear", nominal=TRUE_GEAR, min_value=0.5, max_value=8.0,
        modifier=set_gear_ratio,
    ))
    params["mass"].value[:] = mass_init
    params["gear"].value[:] = gear_init
    return params

### Attempt 1: single sequence (base cart only)

We start at $m = 3.5,\; b = 5.25$, which gives the correct ratio
$b/m = 1.5$ but wrong individual values. Since the cost is exactly zero
everywhere along $b/m = 1.5$, the optimizer has no gradient to follow.

In [ ]:
params_1seq = make_params(mass_init=3.5, gear_init=5.25)

ms_base = sysid.ModelSequences(
    "cart", spec_base, "base_traj", state0_base, ctrl_base, sens_base,
)

residual_fn_1seq = sysid.build_residual_fn(models_sequences=[ms_base])

opt_1seq, result_1seq = sysid.optimize(
    initial_params=params_1seq,
    residual_fn=residual_fn_1seq,
    optimizer="mujoco",
)

print(f"\n--- Single sequence ---")
print(f"  mass:  {opt_1seq['mass'].value[0]:.4f}  (true: {TRUE_MASS})")
print(f"  gear:  {opt_1seq['gear'].value[0]:.4f}  (true: {TRUE_GEAR})")
print(f"  b/m:   {opt_1seq['gear'].value[0] / opt_1seq['mass'].value[0]:.4f}"
      f"  (true: {TRUE_GEAR / TRUE_MASS:.4f})")

The optimizer converged instantly to the **wrong** individual values!
The ratio $b/m$ is correct, but the optimizer has no way to determine $m$ and
$b$ separately.

### Attempt 2: two sequences (base + 1 kg payload)

Adding a second trajectory with a **known 1.0 kg payload** gives the
optimizer a second equation:
- Base:    acceleration $= b / m$
- Payload: acceleration $= b / (m + 1)$

Two equations, two unknowns.

In [ ]:
params_2seq = make_params(mass_init=3.5, gear_init=5.25)

ms_payload = sysid.ModelSequences(
    "cart_payload", spec_pay, "payload_traj", state0_pay, ctrl_pay, sens_pay,
)

residual_fn_2seq = sysid.build_residual_fn(
    models_sequences=[ms_base, ms_payload],
)

opt_2seq, result_2seq = sysid.optimize(
    initial_params=params_2seq,
    residual_fn=residual_fn_2seq,
    optimizer="mujoco",
)

print(f"\n--- Two sequences (base + payload) ---")
print(f"  mass:  {opt_2seq['mass'].value[0]:.4f}  (true: {TRUE_MASS})")
print(f"  gear:  {opt_2seq['gear'].value[0]:.4f}  (true: {TRUE_GEAR})")
print(f"  b/m:   {opt_2seq['gear'].value[0] / opt_2seq['mass'].value[0]:.4f}"
      f"  (true: {TRUE_GEAR / TRUE_MASS:.4f})")

### Visualize the cost landscape

Let's evaluate the cost on a grid of $(m, b)$ values to see the degeneracy.

In [ ]:
#@title compute_cost_surface() { vertical-output: true}
mass_grid = np.linspace(0.6, 4.5, 35)
gear_grid = np.linspace(0.6, 7.5, 35)

def compute_cost_surface(residual_fn, params_template):
    """Evaluate cost on a (mass, gear) grid."""
    cost = np.zeros((len(mass_grid), len(gear_grid)))
    p = params_template.copy()
    for i, m in enumerate(mass_grid):
        for j, g in enumerate(gear_grid):
            x = np.array([m, g])
            res, _, _ = residual_fn(x, p)
            cost[i, j] = sum(np.sum(r**2) for r in res)
    return cost

cost_1seq = compute_cost_surface(residual_fn_1seq, params_1seq)
cost_2seq = compute_cost_surface(residual_fn_2seq, params_2seq)

### Side-by-side cost landscapes

In [ ]:
#@title { vertical-output: true}
G, M = np.meshgrid(gear_grid, mass_grid)

# Compute log cost with shared color range across both panels
log_cost_1 = np.log10(cost_1seq + 1e-12)
log_cost_2 = np.log10(cost_2seq + 1e-12)
vmin = min(log_cost_1.min(), log_cost_2.min())
vmax = max(log_cost_1.max(), log_cost_2.max())
levels = np.linspace(vmin, vmax, 30)

fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True,
                         layout="constrained")

for ax, log_cost, title, opt_p in [
    (axes[0], log_cost_1, "Single sequence", opt_1seq),
    (axes[1], log_cost_2, "Two sequences (base + payload)", opt_2seq),
]:
    cf = ax.contourf(G, M, log_cost, levels=levels, cmap="viridis")

    ax.plot(TRUE_GEAR, TRUE_MASS, "r*", markersize=15, label="True", zorder=5)
    ax.plot(opt_p["gear"].value[0], opt_p["mass"].value[0],
            marker="X", color="gold", markeredgecolor="k", markeredgewidth=1,
            markersize=12, linestyle="none", label="Optimized", zorder=5)

    m_line = np.linspace(0.5, 5.0, 200)
    ax.plot(1.5 * m_line, m_line, "r--", alpha=0.5, lw=1, label="b/m = 1.5")

    ax.set_xlabel("Gear ratio (b)")
    ax.set_title(title)
    ax.grid(True, alpha=0.2)

axes[0].set_ylabel("Mass (m)")

# Shared colorbar
fig.colorbar(cf, ax=axes, label=r"$\log_{10}$(cost)", shrink=0.9)

# Single legend below both plots
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="outside lower center", ncol=3, fontsize=9)

plt.show()

**Left:** With a single sequence, the cost is **exactly zero** along the entire
$b/m = 1.5$ line. The optimizer stays at its starting point because there is no
gradient to follow. Mass and gear are fundamentally unidentifiable.

**Right:** Adding the payload trajectory collapses the valley into a localized
minimum at the true parameter values $(m=2, b=3)$.

**Takeaway:** It is easy to create models with unidentifiable parameters. When this is the case, adding diverse excitations and structural perturbations (e.g., added mass) can resolve it.

# Conclusion

We hope you enjoyed this introduction to system identification with MuJoCo.
The library has additional features not covered here, including:

- **[Physically plausible inertia parameterization](https://ieeexplore.ieee.org/document/9690029)**
  for identifying mass, center of mass, and rotational inertia while
  guaranteeing the result is physically valid
- **Per-sensor weighting** to emphasize certain sensors in the cost
- **Robust loss functions** (Huber, Cauchy, etc.) for handling outliers in
  measured data (see the
  [Least Squares](https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/least_squares.ipynb)
  notebook for details on non-quadratic norms)
- **Multiple optimizer backends** (MuJoCo native, scipy, scipy with parallel
  finite differences)

For a comprehensive survey of the field, see
[Robot Model Identification and Learning: A Modern Perspective](https://www.annualreviews.org/content/journals/10.1146/annurev-control-061523-102310)
(Annual Review of Control, Robotics, and Autonomous Systems, 2024).